# Description
In this notebook, the greeks dataset was also used in the pridiction of sample calsses

In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, confusion_matrix, accuracy_score, log_loss


import sklearn
sklearn.set_config(display="Text")
pd.set_option('display.max_columns', None)

In [2]:
train_path = "../train.csv"
greeks_path = "../greeks.csv"
trainDF = pd.read_csv(train_path)
greeksDF = pd.read_csv(greeks_path)

trainDF.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [56]:
full_trainDF = pd.merge(left=trainDF, right=greeksDF, how="left", on="Id")

X_full, y_full = full_trainDF.iloc[:, :-6], full_trainDF.iloc[:, -6:-1]

X_train_full, X_test, y_train_full, y_test = train_test_split(X_full, y_full, stratify=y_full.iloc[:,0])
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, stratify=y_train_full.iloc[:, 0])

The idea is to have three models;
1. One to learn on the relationship between ordinary and greeks
2. Another to learn the relationship between ordinary and classes
3. The last one to learn the relationship between greek and classes

# Preprocessing

In [10]:
class input_outlier(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        self.feature_names_in = X.columns
        self.upper_limit = X.quantile(0.99)
        self.lower_limit = X.quantile(0.01)
        return self
    def transform(self, X, y=None):
        fixed_X = np.where(X>self.upper_limit, self.upper_limit, np.where(X<self.lower_limit, self.lower_limit, X))
        return fixed_X
    def get_feature_names_out(self, names=None):
        return self.feature_names_in

In [11]:
log_transformer = FunctionTransformer(np.log, feature_names_out="one-to-one")
non_transformer = FunctionTransformer(lambda x: x, feature_names_out="one-to-one")


num_pipeline = make_pipeline(
    input_outlier(),
    SimpleImputer(strategy="median"),
    log_transformer,
    StandardScaler()
)

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)

cleaning_X = ColumnTransformer([
    ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
    ("cat_y", cat_pipeline, ["EJ"]),
])

cleaning_y = ColumnTransformer([
    ("cat_y1", cat_pipeline, ["Alpha", "Beta", "Gamma", "Delta"]),
    #("cat_y2", non_transformer, ["Class"])
])

preprocessing_X = Pipeline([
    ("clean_x", cleaning_X),
    ("poly_feat", PolynomialFeatures(degree=3))
])

preprocessing_y = Pipeline([
    ("clean_y", cleaning_y)
])

In [12]:
X_train_clean = preprocessing_X.fit_transform(X_train)
y_train_clean = preprocessing_y.fit_transform(y_train)

X_valid_clean = preprocessing_X.transform(X_valid)
y_valid_clean = preprocessing_y.transform(y_valid)

X_test_clean = preprocessing_X.transform(X_test)
y_test_clean = preprocessing_y.transform(y_test)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain

log_reg = LogisticRegression(penalty="l2", max_iter=100, class_weight={0:1, 1:8}, tol=0.1, C=7)
chain_clf = ClassifierChain(log_reg, cv=3)
chain_clf.fit(X_train_clean, y_train_clean)

ClassifierChain(base_estimator=LogisticRegression(C=7,
                                                  class_weight={0: 1, 1: 8},
                                                  tol=0.1),
                cv=3)

## Greeks vs Classes

In [65]:
X_data = pd.read_csv(greeks_path)
y_data = pd.read_csv(train_path).iloc[:,-1]

In [72]:
cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)
cleaning_y = ColumnTransformer([
    ("cat_y1", cat_pipeline, ["Alpha", "Beta", "Gamma", "Delta"]),
])